In [1]:
import pandas as pd
import numpy as np
import unicodedata

In [3]:
file_path = "enedis_raw.csv"

df_raw = pd.read_csv(file_path, sep=";", low_memory=False)

df_raw.head()

,annee,code_iris,nom_iris,numero_de_voie,indice_de_repetition,type_de_voie,libelle_de_voie,code_commune,nom_commune,segment_de_client,nombre_de_logements,consommation_annuelle_totale_de_l_adresse_mwh,consommation_annuelle_moyenne_par_site_de_l_adresse_mwh,consommation_annuelle_moyenne_de_la_commune_mwh,adresse,code_epci,code_departement,code_region,tri_des_adresses
0,2018,930100204,Le Moulin à Vent,52.0,NaN,RUE,ETIENNE DOLET,93010,BONDY,RESIDENTIEL,10,17.704,1.770,3.465,52 RUE ETIENNE DOLET,200054781,93.0,11.0,440008
1,2018,930100102,La Terre Saint-Blaise,2.0,NaN,PLACE,EUDEMON,93010,BONDY,RESIDENTIEL,21,53.759,2.560,3.465,2 PLACE EUDEMON,200054781,93.0,11.0,440013
2,2018,930100106,Les Merisiers,211.0,NaN,AVENUE,GALLIENI,93010,BONDY,RESIDENTIEL,168,431.999,2.571,3.465,211 AVENUE GALLIENI,200054781,93.0,11.0,440028
3,2018,930100404,La Ferme,132.0,NaN,AVENUE,GALLIENI,93010,BONDY,RESIDENTIEL,32,33.584,1.049,3.465,132 AVENUE GALLIENI,200054781,93.0,11.0,440031
4,2018,930100302,La Remise à Jorelle,75.0,NaN,AVENUE,GENEVIEVE ANTHONIOZ DE GAULLE,93010,BONDY,RESIDENTIEL,25,152.632,6.105,3.465,75 AVENUE GENEVIEVE ANTHONIOZ DE GAULLE,200054781,93.0,11.0,440038


In [4]:
df_raw.shape

(2821929, 19)

On va garder les colonnes qui semblent les plus utiles

In [5]:
cols = [
    "annee",
    "numero_de_voie",
    #"indice_de_repetition",
    "type_de_voie",
    "libelle_de_voie",
    "code_commune",
    "nom_commune",
    "nombre_de_logements",
    "consommation_annuelle_totale_de_l_adresse_mwh",
    "adresse",
    "code_departement",
    "code_region"
]

df = df_raw[cols].copy()

df.head()

,annee,numero_de_voie,type_de_voie,libelle_de_voie,code_commune,nom_commune,nombre_de_logements,consommation_annuelle_totale_de_l_adresse_mwh,adresse,code_departement,code_region
0,2018,52.0,RUE,ETIENNE DOLET,93010,BONDY,10,17.704,52 RUE ETIENNE DOLET,93.0,11.0
1,2018,2.0,PLACE,EUDEMON,93010,BONDY,21,53.759,2 PLACE EUDEMON,93.0,11.0
2,2018,211.0,AVENUE,GALLIENI,93010,BONDY,168,431.999,211 AVENUE GALLIENI,93.0,11.0
3,2018,132.0,AVENUE,GALLIENI,93010,BONDY,32,33.584,132 AVENUE GALLIENI,93.0,11.0
4,2018,75.0,AVENUE,GENEVIEVE ANTHONIOZ DE GAULLE,93010,BONDY,25,152.632,75 AVENUE GENEVIEVE ANTHONIOZ DE GAULLE,93.0,11.0


In [6]:
df.shape

(2821929, 11)

Le calcul de la consommation par logement en kWh/an

In [7]:
# Passage en numérique
df["nombre_de_logements"] = pd.to_numeric(df["nombre_de_logements"], errors="coerce")
df["consommation_annuelle_totale_de_l_adresse_mwh"] = pd.to_numeric(df["consommation_annuelle_totale_de_l_adresse_mwh"], errors="coerce")

# Au cas ou valeur a 0 ou NaN
mask_valid = df["nombre_de_logements"] > 0

# La division de la conso annuelle de l'adresse par son nombre de logement -> conso par logement
df["conso_par_logement_MWh"] = np.where(mask_valid,df["consommation_annuelle_totale_de_l_adresse_mwh"] / df["nombre_de_logements"],np.nan)

# Conversion en kWh/an
df["conso_par_logement_kWh"] = df["conso_par_logement_MWh"] * 1000

df[["nombre_de_logements", "consommation_annuelle_totale_de_l_adresse_mwh", "conso_par_logement_MWh", "conso_par_logement_kWh"]].head()

,nombre_de_logements,consommation_annuelle_totale_de_l_adresse_mwh,conso_par_logement_MWh,conso_par_logement_kWh
0,10,17.704,1.770400,1770.400000
1,21,53.759,2.559952,2559.952381
2,168,431.999,2.571423,2571.422619
3,32,33.584,1.049500,1049.500000
4,25,152.632,6.105280,6105.280000


On essaie de formaliser les adresses dans un style : 

"numero_de_voie + type_de_voie + libelle_de_voie + nom_commune"

In [8]:
def normalize_text(s: str) -> str:
    if pd.isna(s):
        return ""
    # passage en str
    s = str(s)
    # enlever accents
    s = unicodedata.normalize('NFKD', s)
    s = "".join(c for c in s if not unicodedata.combining(c))
    # majuscules
    s = s.upper()
    # remplace toute ponctuation par espace
    for ch in [",", ";", ":", ".", "/", "\\", "'", '"', "(", ")", "-", "_"]:
        s = s.replace(ch, " ")
    # espaces en trop
    s = " ".join(s.split())
    return s

In [9]:
# Nettoyage du numéro de voie
df["numero_de_voie_num"] = (pd.to_numeric(df["numero_de_voie"], errors="coerce").round().astype("Int64"))

# On transforme en string propre pour la concaténation
num = df["numero_de_voie_num"].astype("object").fillna("").astype(str)

# Nettoyage du type de voie 
df["type_de_voie"] = df["type_de_voie"].astype(str)
df["type_de_voie"] = df["type_de_voie"].replace(["0", "0.0", "nan", "None", "NONE", "NaN"], "")
typ = df["type_de_voie"].fillna("").astype(str)

# Nettoyage du libelle de voie
nom_voie = df["libelle_de_voie"].fillna("").astype(str)

# Nettoyage du nom de commune
commune = df["nom_commune"].fillna("").astype(str)

# Construction
adresse_brut_concat = num + " " + typ + " " + nom_voie + " " + commune

# Normalisation 
df["adresse_clean"] = adresse_brut_concat.apply(normalize_text)

df[["adresse", "adresse_clean"]].head(10)

,adresse,adresse_clean
0,52 RUE ETIENNE DOLET,52 RUE ETIENNE DOLET BONDY
1,2 PLACE EUDEMON,2 PLACE EUDEMON BONDY
2,211 AVENUE GALLIENI,211 AVENUE GALLIENI BONDY
3,132 AVENUE GALLIENI,132 AVENUE GALLIENI BONDY
4,75 AVENUE GENEVIEVE ANTHONIOZ DE GAULLE,75 AVENUE GENEVIEVE ANTHONIOZ DE GAULLE BONDY
5,73 AVENUE GENEVIEVE ANTHONIOZ DE GAULLE,73 AVENUE GENEVIEVE ANTHONIOZ DE GAULLE BONDY
6,2 ALLEE GEORGE SAND,2 ALLEE GEORGE SAND BONDY
7,40 PLACE GEORGES LYSSANDRE,40 PLACE GEORGES LYSSANDRE BONDY
8,20 RUE GUILLAUME APOLLINAIRE,20 RUE GUILLAUME APOLLINAIRE BONDY
9,46 AVENUE HENRI BARBUSSE,46 AVENUE HENRI BARBUSSE BONDY


In [10]:
nb_diff = (df["adresse"].apply(normalize_text) != df["adresse_clean"]).sum()
pct_diff = nb_diff / len(df) * 100
pct_diff
print("Nombre d'adresse modifié : ", nb_diff, "soit", pct_diff, "%") 

Nombre d'adresse modifié :  2821872 soit 99.99798010509832 %


In [11]:
df.head()

,annee,numero_de_voie,type_de_voie,libelle_de_voie,code_commune,nom_commune,nombre_de_logements,consommation_annuelle_totale_de_l_adresse_mwh,adresse,code_departement,code_region,conso_par_logement_MWh,conso_par_logement_kWh,numero_de_voie_num,adresse_clean
0,2018,52.0,RUE,ETIENNE DOLET,93010,BONDY,10,17.704,52 RUE ETIENNE DOLET,93.0,11.0,1.770400,1770.400000,52,52 RUE ETIENNE DOLET BONDY
1,2018,2.0,PLACE,EUDEMON,93010,BONDY,21,53.759,2 PLACE EUDEMON,93.0,11.0,2.559952,2559.952381,2,2 PLACE EUDEMON BONDY
2,2018,211.0,AVENUE,GALLIENI,93010,BONDY,168,431.999,211 AVENUE GALLIENI,93.0,11.0,2.571423,2571.422619,211,211 AVENUE GALLIENI BONDY
3,2018,132.0,AVENUE,GALLIENI,93010,BONDY,32,33.584,132 AVENUE GALLIENI,93.0,11.0,1.049500,1049.500000,132,132 AVENUE GALLIENI BONDY
4,2018,75.0,AVENUE,GENEVIEVE ANTHONIOZ DE GAULLE,93010,BONDY,25,152.632,75 AVENUE GENEVIEVE ANTHONIOZ DE GAULLE,93.0,11.0,6.105280,6105.280000,75,75 AVENUE GENEVIEVE ANTHONIOZ DE GAULLE BONDY


In [12]:
df.to_csv("enedis_clean.csv", index=False, encoding="utf-8")